In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

from RIS_gravity_inversion import regional, synthetic, plotting, optimization
import RIS_gravity_inversion.inversion as inv
import RIS_gravity_inversion.utils as inv_utils
import RIS_gravity_inversion.gravity_processing as grav

import copy
from antarctic_plots import utils, regions, maps, fetch, profile
import zarr
import harmonica as hm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import verde as vd
import scipy as sp
import pvxarray
import rioxarray
import pygmt
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import plotly
import plotly.figure_factory as ff
import seaborn as sns

sns.set_theme()
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
import warnings
import itertools
import pickle
import random


import os

# make beep sound when notify is called at end of long-running cells
from IPython.display import Audio

wave = np.sin(2 * np.pi * 400 * np.arange(10000 * 1) / 10000)
notify = Audio(wave, rate=10000, autoplay=True)

In [2]:
plot = True
# plot = False
runs = 50

In [3]:
# choose resolution

layer_spacing = 5e3
# layer_spacing = 10e3

grav_spacing = 5e3
# grav_spacing = 10e3

# Load inversion inputs

In [4]:
# weights grid and starting bathymetry
starting_prisms = xr.open_zarr(
    f"../inversion_layers/RIS_starting_prisms_{int(layer_spacing)}k_constraints.zarr"
)

# partial bouguer corrected gravity data
df_anomalies = pd.read_csv(
    "../data/RIS_residual_50k_filt_mask.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# set regions
buffer_region = utils.get_grid_info(starting_prisms.density)[1]
inversion_region = vd.get_region((df_anomalies.easting, df_anomalies.northing))

# set layer parameters
registration = utils.get_grid_info(starting_prisms.density)[4]

print(f"layer spacing: {layer_spacing}m")
print(f"registration: {registration}")
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

if layer_spacing == 5e3:
    fname = "../inversion_layers/bed_from_constraints_5k.zarr"
elif layer_spacing == 10e3:
    fname = "../inversion_layers/bed_from_constraints_10k.zarr"
bed_from_constraints = xr.open_zarr(fname).bed
bed_from_constraints = fetch.resample_grid(
    bed_from_constraints,
    region=buffer_region,
    registration=registration,
    verbose="q",
).rename({"x": "easting", "y": "northing"})

# constraints
if layer_spacing == 5e3:
    constraints_fname = "../data/bed_constraints_5k.csv.gz"
elif layer_spacing == 10e3:
    constraints_fname = "../data/bed_constraints_10k.csv.gz"
else:
    raise ValueError("No constraint point files available at specified spacing!")

constraints = pd.read_csv(
    constraints_fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# mark points inside grounding line as "inside"
measures_shelves = fetch.measures_boundaries(version="IceShelf")
ice_shelves = gpd.read_file(measures_shelves)
RIS_seperate = ice_shelves[ice_shelves.NAME.isin(["Ross_West", "Ross_East"])]
RIS = RIS_seperate.dissolve()
gdf = gpd.GeoDataFrame(
    constraints,
    geometry=gpd.points_from_xy(x=constraints.easting, y=constraints.northing),
    crs="EPSG:3031",
)
constraints["inside"] = gdf.within(RIS.geometry[0])
constraints.drop(columns="geometry", inplace=True)

# ensure all points are within buffer region
constraints = utils.points_inside_region(
    constraints, buffer_region, names=("easting", "northing")
)
print(f"bathymetry control points:{len(constraints)}")

# surface
fname = "../inversion_layers/bedmachine_surface.zarr"
surface_full = xr.open_zarr(fname).surface
surface = fetch.resample_grid(
    surface_full,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
surface_layer_spacing = fetch.resample_grid(
    surface_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
# icebase
fname = "../inversion_layers/bedmachine_icebase.zarr"
icebase_full = xr.open_zarr(fname).icebase
icebase = fetch.resample_grid(
    icebase_full,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
icebase_layer_spacing = fetch.resample_grid(
    icebase_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)

# bed
bedmachine_bed = xr.open_zarr("../inversion_layers/bedmachine_bed.zarr")["bed"]
bedmachine_bed_layer_spacing = fetch.resample_grid(
    bedmachine_bed,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)

# final inverted bathymetry model
fname = "../results/weights_mask_prism_results.csv.gz"
inverted_bathy = pd.read_csv(
    fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
inverted_bathy = inverted_bathy.set_index(["northing", "easting"]).to_xarray()[
    inverted_bathy.columns[-1]
]

/home/tankerma/miniconda/envs/RIS_gravity_inversion/lib/python3.10/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


layer spacing: 5000.0m
registration: g
inversion region =  (-580000.0, 385000.0, -1385000.0, -445000.0)
buffer region =  [-620000.0, 460000.0, -1460000.0, -380000.0]
bathymetry control points:28087
spacing larger than original, filtering and resampling
spacing larger than original, filtering and resampling
spacing larger than original, filtering and resampling


# Add uncertainties 
2.9 mGal for gravity data

5% depth from ice surface for constraints within ice shelf

In [5]:
# sample ice surface elevation into dataframe
# from bedmachine,
constraints = profile.sample_grids(
    constraints,
    surface,
    "surface_z_ellipsoidal",
    coord_names=["easting", "northing"],
)

# calculate error as 5% of depth from surface
constraints["z_error"] = 10
depth = (
    constraints[constraints.inside].surface_z_ellipsoidal
    - constraints[constraints.inside].upward
)
uncert = 0.05 * depth

constraints.loc[constraints[constraints.inside].index, "z_error"] = uncert

constraints["low_lim"] = constraints.upward - constraints.z_error
constraints["high_lim"] = constraints.upward + constraints.z_error

print(
    f"mean uncertainty inside RIS: {np.nanmean(constraints[constraints.inside].z_error)} m"
)
print(
    f"mean uncertainty outside RIS: {np.nanmean(constraints[constraints.inside==False].z_error)} m"
)
constraints.describe()

mean uncertainty inside RIS: 33.46741474651805 m
mean uncertainty outside RIS: 10.0 m


,northing,easting,upward,lat,lon,station_ID,z_geoidal,z_error,measurement_type,surface_z_ellipsoidal,low_lim,high_lim
count,2.808700e+04,28087.000000,28087.000000,223.000000,223.000000,223.000000,0.0,28087.000000,0.0,28087.000000,28087.000000,28087.000000
mean,-8.756051e+05,-90456.515288,-18.108343,-80.352235,-9.656647,13.699552,NaN,10.186322,NaN,823.526287,-28.294665,-7.922021
std,3.686328e+05,357858.552976,1030.160378,1.687297,168.173939,8.608311,NaN,2.207073,NaN,1002.110461,1030.291970,1030.033498
min,-1.460000e+06,-620000.000000,-3737.661133,-84.600000,-179.819200,1.000000,NaN,10.000000,NaN,-61.000000,-3747.661133,-3727.661133
25%,-1.280917e+06,-425000.000000,-651.421906,-81.603470,-165.579050,1.000000,NaN,10.000000,NaN,-54.000000,-661.601593,-641.189209
50%,-7.550000e+05,-120000.000000,-452.590332,-79.976110,-154.674400,19.000000,NaN,10.000000,NaN,338.849823,-462.769196,-442.387787
75%,-5.450000e+05,270000.000000,566.328583,-79.044165,170.377200,19.000000,NaN,10.000000,NaN,1704.700073,556.328583,576.328583
max,-3.800000e+05,460000.000000,4053.667969,-77.656670,179.950300,25.000000,NaN,69.764264,NaN,4220.422852,4043.667969,4063.667969


In [6]:
df_anomalies["uncert"] = 2.9
df_anomalies

,northing,easting,Gobs,upward,surface_forward,icebase_forward,Gobs_corr,bed_forward,Gobs_corr_shift,misfit,reg,res,uncert
0,-1385000.0,55000.0,-5.917760,1000.0,-20.684952,0.044324,14.722868,-18.890848,-19.301201,-0.410353,-0.410353,-2.024619e-08,2.9
1,-1385000.0,60000.0,-3.988423,1000.0,-20.665307,0.046457,16.630428,-18.311723,-17.393641,0.918081,0.918081,-3.300273e-08,2.9
2,-1380000.0,45000.0,-10.974624,1000.0,-20.701629,0.042514,9.684491,-21.866672,-24.339578,-2.472906,-2.472906,3.934122e-08,2.9
3,-1380000.0,50000.0,-7.966111,1000.0,-20.691564,0.043529,12.681924,-21.290713,-21.342146,-0.051433,-0.051433,1.256685e-07,2.9
4,-1380000.0,55000.0,-5.154862,1000.0,-20.669303,0.045952,15.468489,-20.048731,-18.555581,1.493150,1.493150,1.397200e-08,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22236,-445000.0,-210000.0,-26.054879,1000.0,-15.087119,0.030805,-10.998566,10.640792,-45.022635,-55.663427,-55.663425,-1.764548e-06,2.9
22237,-445000.0,-205000.0,-25.186605,1000.0,-11.738032,0.551109,-13.999682,16.649674,-48.023752,-64.673426,-64.673424,-1.975152e-06,2.9
22238,-445000.0,-200000.0,-22.535298,1000.0,-1.044918,2.062803,-23.553182,32.654473,-57.577251,-90.231725,-90.231728,2.897721e-06,2.9
22239,-445000.0,-195000.0,-18.995592,1000.0,-3.329588,1.836528,-17.502531,32.284136,-51.526601,-83.810736,-83.810738,1.209740e-06,2.9


# Setup Monte-Carlo sampling
* define sampling methods for the uncertainties of both the gravity and constraints data
* run full inversion with sampled gravity
    * including: 
        * corrections for parital bouguer and starting bed
        * DC shift
        * regional / residual seperation
        * inversion
* find standard deviation per grid cell of all resulting inverted bathymetry models

## Run inversion workflow

In [19]:
# get RIS mask
measures_shelves = fetch.measures_boundaries(version="IceShelf")
ice_shelves = gpd.read_file(measures_shelves)
RIS_seperate = ice_shelves[ice_shelves.NAME.isin(["Ross_West", "Ross_East"])]
RIS = RIS_seperate.dissolve()

bedmachine_df_buffer, _, _ = inv_utils.inversion_buffer_points(
    buffer_width=20e3,
    grid=bedmachine_bed_layer_spacing.rename({"x": "easting", "y": "northing"}),
    mask=RIS,
    # plot = True,
)
bedmachine_df_buffer

,northing,easting,upward
3843,-1375000.0,150000.0,-675.252319
3844,-1375000.0,155000.0,-693.543945
3845,-1375000.0,160000.0,-727.819214
3846,-1375000.0,165000.0,-740.022217
3847,-1375000.0,170000.0,-731.086975
...,...,...,...
45208,-420000.0,-260000.0,-96.547409
45209,-420000.0,-255000.0,-503.788727
45210,-420000.0,-250000.0,-459.763702
45424,-415000.0,-265000.0,290.040894


In [ ]:
kwargs = {
    "grav_spacing": grav_spacing,
    "inversion_region": inversion_region,
    "buffer_region": buffer_region,
    "layer_spacing": layer_spacing,
    # optional:
    "weights_grid": starting_prisms.weights,
    "starting_bed_prisms": starting_prisms,
    "starting_bed": bed_from_constraints,
    "icebase_lowres": icebase_layer_spacing.rename({"x": "easting", "y": "northing"}),
    "icebase_fullres": icebase,
    "surface_fullres": surface,
    "bed_buffer_points": bedmachine_df_buffer,
}
inversion_args = {
    "input_grav_column": "Gobs_corr_shift",
    "apply_weights": False,
    # "apply_weights": True,
    "weights_after_solving": True,
    "max_iterations": 20,
    "l2_norm_tolerance": 0.2,
    "delta_l2_norm_tolerance": 0.8,
    "perc_increase_limit": 0.2,
    "upper_confining_layer": icebase_layer_spacing,
    "deriv_type": "annulus",
    "solver_type": "scipy least squares",
}

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_full.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    sample_tension_factor=True,  # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    sample_solver_damping=True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Full Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_gravity.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Gravity Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_constraints.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Constraints Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_tension.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    sample_tension_factor=True,  # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Tension factor Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_ice_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (ice) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_water_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (water) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_sediment_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (sediment) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_inner_bound_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds (inner) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_outer_bound.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds (outer) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_solver_damping.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    sample_solver_damping=True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Damping Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_data.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Data Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_densities.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Densities Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_bounds.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_full_workflow_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
# residuals = [utils.RMSE(df[list(df.columns)[-1]]) for df in grav_dfs]
# stats = inv_utils.model_ensemble_stats(topos, weights = residuals)
# z_stdev, weighted_stdev = stats[2], stats[4]
# grd = z_stdev - weighted_stdev
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp",
#                  xr_grid = grd, masked=True, invert=False)
# maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="weighted standard deviation (m)",
#     # title=title,
# )

## Calculate standard deviation
* calculate standard deviation of each grid cell for the entire suite of invert bathymetries

In [ ]:
# values_df = pd.DataFrame(values)
# fig, axes = plt.subplots(1,3)
# sns.histplot(ax=axes[0], data=values_df, x="ice_density", kde=True)
# sns.histplot(ax=axes[1], data=values_df, x="water_density", kde=True)
# sns.histplot(ax=axes[2], data=values_df, x="sediment_density", kde=True)

In [ ]:
# grd = ds_merged['run_num'].std("runs").rename("z_std")
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = grd, masked=True, invert=False)

# fig = maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="standard deviation (m)",
#     title='Combined Monte Carlo uncertainty',
# )

# fig.plot(
#     x=constraints.easting,
#     y=constraints.northing,
#     fill="black",
#     style="c.01c",
# )
# fig.plot(
#     x=constraints[constraints.inside].easting,
#     y=constraints[constraints.inside].northing,
#     fill="white",
#     pen=".3p,black",
#     style="c.08c",
#     label="Bed constraints",
# )
# fig.legend()

# fig.show()

In [ ]:
# grd = ds_merged['run_num'].median("runs").rename("z_median")
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = grd, masked=True, invert=False)

# fig = maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="median (m)",
#     title='Combined Monte Carlo median',
# )

# fig.plot(
#     x=constraints.easting,
#     y=constraints.northing,
#     fill="black",
#     style="c.01c",
# )
# fig.plot(
#     x=constraints[constraints.inside].easting,
#     y=constraints[constraints.inside].northing,
#     fill="white",
#     pen=".3p,black",
#     style="c.08c",
#     label="Bed constraints",
# )
# fig.legend()

# fig.show()

In [ ]:
# merged_min = ds_merged['run_num'].min("runs")
# merged_max = ds_merged['run_num'].max("runs")
# merged_min = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = merged_min, masked=True, invert=False)
# merged_max = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = merged_max, masked=True, invert=False)

# _ = utils.grd_compare(
#     merged_min,
#     merged_max,
#     plot=True,
#     grid1_name="Min",
#     grid2_name="Max",
#     title='Combined Monte Carlo analysis',
#     cbar_label="variance (m)",
#     diff_cmap="inferno",
#     diff_maxabs=False,
#     cmap="rain",
#     reverse_cpt=True,
#     rmse_in_title=False,
#     subplot_labels=True,
#     robust=True,
#     hist=True,
#     # shp_mask="../plotting/RIS_outline.shp",
# )

In [ ]:
# inverted_bathy_mask = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = inverted_bathy, masked=True, invert=False)
# max_to_final = merged_max - inverted_bathy_mask
# min_to_final = merged_min - inverted_bathy_mask

# fig = maps.plot_grd(
#     max_to_final,
#     title="Max variance - final model",
#     cmap="vik+h0",
#     robust=True,
#     hist=True,
# )
# fig = maps.plot_grd(
#     min_to_final,
#     title="Min variance - final model",
#     cmap="vik+h0",
#     robust=True,
#     hist=True,
#     fig = fig,
#     origin_shift = "xshift",
# )
# fig.show()